In [3]:
import numpy as np
import sys, os

In [4]:
sys.path.append("..")
from qgbaroclinic import OceBaroclinicModes
from qgbaroclinic.plot import PlotMap

In [ ]:
"""
Example for computing the MAP of rossby radii in a selected region.
"""

In [6]:
# Define Ocean Baroclinic Modes object;
# "latitude", "longitude" are the dimension names in OCEAN NetCDF file.
obm = OceBaroclinicModes(longitude=[-30, -28], latitude=[40, 42])

In [7]:
# Extract OCEAN variables form NetCDF file.
current_path = os.path.dirname(__vsc_ipynb_file__)
temp, sal, depth, lat , lon = obm.read(
    os.path.join(current_path, "../data/azores_Jan2021.nc"),
    "thetao",
    "so",
    "depth",
    "latitude",
    "longitude",
)

In [ ]:
elevation = np.ones_like(temp) * 3000 # dummy elevation

In [10]:
# Compute Potential Density and average in time.
pot_density = obm.potential_density(temp, sal)
mean_pot_density = np.mean(pot_density, axis = 0)

In [11]:
n_modes = 3
lat_size, lon_size = mean_pot_density.shape[1:]
rossby_rad = np.empty([lat_size, lon_size, n_modes])

In [ ]:
# Compute Rossby Baroclinic Deformation Radii.
for i in range(lat_size):
    for j in range(lon_size):
        try:
            rossby_rad[i,j,:], _ = obm(mean_pot_density[:,i,j].values, depth.values, lat[i].values, elevation[i,j])
        except ValueError:
            pass

In [3]:
# Plot and compare with Chelton (1998)
pm = PlotMap([-30, -28], [40, 42], ".")
pm.make_plot(rossby_rad[:,:,1]/1000,lon, lat, "2d_map_example")
pm.plot_chelton_map()